In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ast
import cv2

In [2]:
# img = cv2.resize('WBL-logos_transparent.png', fx=0.25, fy=0.25)
# cv2.imwrite('WBL_resized',img)

In [3]:
src = cv2.imread('WBL-logos_transparent.png')
scale_percent = 50
width = int(src.shape[1] * scale_percent)
height = int(src.shape[0] * scale_percent)
dsize = (width, height)

# resize image
output = cv2.resize(src, dsize)

cv2.imwrite('WBL-logos_resized3_transparent.png',output) 

True

In [4]:
whatsapp = pd.read_csv('Recommender system (2).csv')

In [5]:
whatsapp.head()

,Number,Business_Name,Business_Type,Business_Address,Business_description,Business_link,Business_email,Business_products
0,2348065202633,Kemi,Grocery Store,"Odo oshi street, Idi ogun, Ijare. Akure, ondo ...","Odo oshi street, Idi ogun, Ijare. Akure, ondo ...",https://api.whatsapp.com/send/?phone=234806520...,adigunhaleematoluwakemi@gmail.com,Ankara
1,2349033244466,X CONCEPT MEDIA NIG LTD,DJ • Photography Videography • Music Productio...,"NO 38, UCHE ONYUKWU CRESCENT, ABUJA","Born out of dreams, we recognize the importanc...",https://api.whatsapp.com/send/?phone=234903324...,xconceptmedia@gmail.com,Dj
2,2347011929615,Pafet Ltd,Printing Service,"No 60, Bauchi Ring Road,, Jos, Nigeria",Feel free to get in touch with us. We have exp...,https://api.whatsapp.com/send/?phone=234701192...,pafetimage@gmail.com,CAR TISSUE
3,2348100000465,Maison Loulabelle,Shopping & retail • Apparel & clothing,"19 Alexandria Cres, Wuse, Abuja, Nigeria",Africa's leading luxury childrenswear brand,https://api.whatsapp.com/send/?phone=234810000...,maisonloulabelle@gmail.com,Oyiza lace up back dress
4,2348100000018,One Day........,"Beauty, cosmetic & personal care",Kwari Market Gidan Nababa Badamasi Opp First Bank,Bags & Shoes,https://api.whatsapp.com/send/?phone=234810000...,www.muhammadsani983@gmail.com,Bags & Shoes


In [6]:
whatsapp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Number                28 non-null     int64 
 1   Business_Name         28 non-null     object
 2   Business_Type         28 non-null     object
 3   Business_Address      26 non-null     object
 4   Business_description  27 non-null     object
 5   Business_link         28 non-null     object
 6   Business_email        21 non-null     object
 7   Business_products     28 non-null     object
dtypes: int64(1), object(7)
memory usage: 1.9+ KB


In [7]:
whatsapp['Business_Address']

0     Odo oshi street, Idi ogun, Ijare. Akure, ondo ...
1                   NO 38, UCHE ONYUKWU CRESCENT, ABUJA
2                No 60, Bauchi Ring Road,, Jos, Nigeria
3              19 Alexandria Cres, Wuse, Abuja, Nigeria
4     Kwari Market Gidan Nababa Badamasi Opp First Bank
5      1a Ozumba Mbadiwe Victoria Island Lagos, Nigeria
6     1 Papa Road, Toba Shopping Complex, Beside Aga...
7     1st Floor Cancer Block, Eleganza Plaza, Wharf ...
8     KM 46 Lekki - Epe Expy, opposite SBI hotel, Et...
9     Seyisoge_makeovers Contact us for your:Face gl...
10           Civic Center, Mountain View, CA 94041, USA
11                          Catherine China importation
12                                 Education Strategist
13                                                  NaN
14    No 3,Barrister Olaolu Shyllon Street, New road...
15    No 50,Liberty road,off ring road, beside Okat ...
16    My name is Mr Tochi.I'm an Educational consult...
17                                We Deliver Wor

In [8]:
whatsapp.columns

Index(['Number', 'Business_Name', 'Business_Type', 'Business_Address',
       'Business_description', 'Business_link', 'Business_email',
       'Business_products'],
      dtype='object')

In [9]:
whatsapp['Business_Name'].isnull().sum()

0

In [10]:
whatsapp['Business_Type'].isnull().sum()

0

In [11]:
whatsapp['Business_Address'].isnull().sum()

2

In [12]:
whatsapp['Business_description'].isnull().sum()

1

In [13]:
vectorizer = TfidfVectorizer(stop_words='english')

In [14]:
word_vectors = vectorizer.fit_transform(whatsapp['Business_Type'])

In [15]:
# whatsapp = whatsapp.set_index('Number')

In [16]:
indicies= pd.Series(whatsapp.index, index = whatsapp['Business_Type'])

In [17]:
print(indicies)

Business_Type
Grocery Store                                               0
DJ • Photography Videography • Music Production Studio      1
Printing Service                                            2
Shopping & retail • Apparel & clothing                      3
Beauty, cosmetic & personal care                            4
Apparel & clothing                                          5
Beauty, cosmetic & personal care                            6
Cargo & Freight Company                                     7
Education                                                   8
Beauty, cosmetic & personal care                            9
Social Media Agency                                        10
Shopping & retail                                          11
Education                                                  12
Education                                                  13
Photographer • Camera/photo • Performance & Event Venue    14
Arts & entertainment • Event Planner                    

In [18]:
word_vectors.shape

(28, 40)

In [19]:
similarity_scores = cosine_similarity(word_vectors)

In [20]:
similarity_scores.shape

(28, 28)

In [21]:
indicies = indicies.groupby(indicies.index).first()

In [22]:
def recommend(m_des, similarity_scores, k = 5):
    m_id = indicies[m_des]
    similar_item_scores = list(enumerate(similarity_scores[m_id]))
    sorted_sim_scores = sorted(similar_item_scores, key = lambda x:x[1], reverse = True)[1:k+1]
    m_index = [idx[0] for idx in sorted_sim_scores]
    r_whatsapp = whatsapp.iloc[m_index]['Business_Type']
    r_numbers = whatsapp.iloc[m_index]['Number']
    r_address = whatsapp.iloc[m_index]['Business_Address']
#     result = whatsapp.iloc[m_index]['Number']
#     result = whatsapp
    Biz_list = [r_whatsapp[x] for x in m_index]
    Num_list = [r_numbers[x] for x in m_index]
    add_list = [r_address[x] for x in m_index]
    
    final_series = pd.DataFrame(index = [Num_list, add_list], data = Biz_list)
#     print(Num_list)
    return final_series

In [23]:
recommend('Beauty, cosmetic & personal care', similarity_scores)

,,0
2348132084652,"1 Papa Road, Toba Shopping Complex, Beside Agarahu Complex Oju-ore Ota","Beauty, cosmetic & personal care"
2347053347615,"Seyisoge_makeovers Contact us for your:Face glam and Gele tying, Training Makeup products","Beauty, cosmetic & personal care"
2348052673193,We Deliver World Wide,"Beauty, cosmetic & personal care"
2348065202633,"Odo oshi street, Idi ogun, Ijare. Akure, ondo state",Grocery Store
2349033244466,"NO 38, UCHE ONYUKWU CRESCENT, ABUJA",DJ • Photography Videography • Music Productio...


In [24]:
whatsapp.to_csv('whatsweb.csv')

In [25]:
whatsapp['Business_Type']

0                                         Grocery Store
1     DJ • Photography Videography • Music Productio...
2                                      Printing Service
3                Shopping & retail • Apparel & clothing
4                      Beauty, cosmetic & personal care
5                                    Apparel & clothing
6                      Beauty, cosmetic & personal care
7                               Cargo & Freight Company
8                                             Education
9                      Beauty, cosmetic & personal care
10                                  Social Media Agency
11                                    Shopping & retail
12                                            Education
13                                            Education
14    Photographer • Camera/photo • Performance & Ev...
15                 Arts & entertainment • Event Planner
16                                            Education
17                     Beauty, cosmetic & person

In [26]:
whatsapp['Business_products']

0                                                Ankara
1                                                    Dj
2                                            CAR TISSUE
3                              Oyiza lace up back dress
4                                          Bags & Shoes
5                        Quoted shoulder bag with chain
6                                         BRIDAL MAKEUP
7                               Import and Export Agent
8                              *TOFEL TUTORIAL ACADEMY*
9                                             Brush set
10                              YouTube Content creator
11                                          Oil perfume
12                            Essential School Policies
13                                               School
14                     Weddings/Portraits /home service
15    Frame,wall clock,nylon,paper bag,stickers,mock...
16                                      CODING FOR KIDS
17                                              

In [27]:
whatsapp[6:10]

,Number,Business_Name,Business_Type,Business_Address,Business_description,Business_link,Business_email,Business_products
6,2348132084652,Lizsmile beauty lounge,"Beauty, cosmetic & personal care","1 Papa Road, Toba Shopping Complex, Beside Aga...","Welcome to Lizsmile beauty lounge, your one-st...",https://api.whatsapp.com/send/?phone=234813208...,Salaamoye9@gmail.com,BRIDAL MAKEUP
7,2347030604506,Joakema International Limited,Cargo & Freight Company,"1st Floor Cancer Block, Eleganza Plaza, Wharf ...",*Customer's Satisfaction is our Utmost Priority*,https://api.whatsapp.com/send/?phone=234703060...,joakemainternational@yahoo.com,Import and Export Agent
8,2347035556120,Philarboi,Education,"KM 46 Lekki - Epe Expy, opposite SBI hotel, Et...","We offer LECTURES & REGISTRATIONS for JAMB, WA...",https://api.whatsapp.com/send/?phone=234703555...,proffelix10@gmail.com,*TOFEL TUTORIAL ACADEMY*
9,2347053347615,Seyisoge Makeovers,"Beauty, cosmetic & personal care",Seyisoge_makeovers Contact us for your:Face gl...,Seyisoge_makeovers Contact us for your:Face gl...,https://api.whatsapp.com/send/?phone=234705334...,NaN,Brush set


In [28]:
pd.DataFrame()

""
